## Google Drive

These are things you only need to do once to connect all the data pipelines.

(*Skip these steps if this is open in a colab notebook in your Google Drive.*)

1.) open this notebook up in Google Colab:

https://colab.research.google.com/github/Ready4theCrush/concha/blob/master/notebooks/02_setup_do_once.ipynb

2.) Go to "File" -> "Save a copy in Drive"
This makes your own copy in you drive in a folder called "Colab Notebooks".

3.) Connect to Google Drive to save your work by running the next cell.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Run this cell first to bring in the concha code
!pip -q install concha
from concha import Planner

## Square

Concha learns from how each product sold to determine how much to make in the future. That history is saved
by Square.

1.) Activate yourself as a developer here: https://developer.squareup.com/apps.

2.) Create an "app", you can call it anything you want.

3.) Click "Open" on the card shaped icon for the "Application" you just created.

4.) On the menu on left side, click on "OAuth".

5.) Set "Production Redirect URL" to "http://www.google.com"

6.) Copy and paste the "Production Application ID" and the "Production Application Secret" (Click "Show"
    first) into the next cell.

The app you are creating is like a data "agent" to pull detailed records of exactly when each item was ordered each day. The "Application ID" and "Application Secret" are for the "agent". In the next cell 
you are granting the agent (limited) access to your own sales history. It requests access to read order histories ("ORDERS READ") and store locations ("MERCHANT PROFILE READ"). Setting the redirect URL to www.google.com
is just a trick to get the code Square creates once you grant access.

In [2]:
#@title Square Application Information

production_application_id = "" #@param {type:"string"}
production_application_secret = "" #@param {type:"string"}

from concha.importers import Square
sqr = Square()
request_url = sqr.create_access_request_url(production_application_id, production_application_secret)
print("Copy this url into a browswer to request data access for the agent you created: \n")
print(request_url)

Copy this url into a browswer to request data access for the agent you created: 

https://connect.squareup.com/oauth2/authorize?client_id=sq0idp-FRq67m8D9NyTbO8sOblH1Q&scope=ORDERS_READ+MERCHANT_PROFILE_READ


After going to the url printed out above, you should be redirected to the google homepage, except that the authorization code is now listed in the url. Copy the
new url, paste it below and run the cell to get the access token.

In [3]:
#@title Square Application Information

redirect_url = "" #@param {type:"string"}
response = sqr.get_access_token(redirect_url)
response.body

<ApiResponse [{
  "access_token": "EAAAEGq5qB3yf1-HieRTGdM25TIqXcTkqb8xJko5_7SW36wmoluXlxFXJWeA44bn",
  "token_type": "bearer",
  "expires_at": "2020-12-12T22:13:51Z",
  "merchant_id": "MLCSFP53YAJA6",
  "refresh_token": "EQAAELrHzSYKxv1sK2ZCRzGTGTjWZN-x-l5FjDlcZmugw5kfs3aUkJhAe4dWtm6D"
}]>

If you see an "access_token" in the response, then it worked!

Why go through all this trouble to create an access token instead of just using the "Production Access Token provided on the developer website? Because the token we created is safer: it only allows access for reading orders and business locations. The one on the website gives access to everything.

The new Square credentials are stored on your drive in:
`/content/drive/My Drive/concha_planners/importers/square.json`

In [5]:
# Now let's see the list of vendor locations.
locs = sqr.get_locations()
locs

{'Center City': {'id': 'L074SHENBT3WK',
  'name': 'Center City',
  'address': '275 S Raymond Ave, CA, US, Pasadena, 91105',
  'lat': 34.141888,
  'lng': -118.149274,
  'timezone': 'America/Los_Angeles'},
 'Concha West': {'id': 'LPZBW2H2WPS6M',
  'name': 'Concha West',
  'address': '415 S Arroyo Blvd, CA, US, Pasadena, 91105',
  'lat': 34.13827779625123,
  'lng': -118.1661243381397,
  'timezone': 'America/Los_Angeles'}}

## Weather

To look up the past weather conditions on each day, you need an API key for NOAA.

1.) Go here: https://www.ncdc.noaa.gov/cdo-web/token and put in your email.

2.) Enter the API key they send you in the next cell.

In [14]:
#@title NOAA (Weather) API Key

noaa_api_key = "" #@param {type:"string"}

from concha.weather import NOAA
weather = NOAA()
weather.set_api_key(noaa_api_key)

## Check Everything

Let's make a planner to make sure it all worked.

In [12]:
from concha import Planner
example = Planner(planner_name="example")
example.set_importer(sqr)
first_location = list(sqr.settings['locations'].keys())[0]
example.set_location(first_location)
example.set_weather_agent(weather)
example.set_weather_station()
forecast = example.generate_daily_forecast_metadata()
forecast

location for example set to Center City.


,date,day_of_week,tmin,tmax,prcp,snow
0,2020-11-13,Fri,45.0,70.0,False,False
1,2020-11-14,Sat,47.0,73.0,False,False
2,2020-11-15,Sun,48.0,81.0,False,False
3,2020-11-16,Mon,52.0,89.0,False,False
4,2020-11-17,Tue,55.0,83.0,False,False
5,2020-11-18,Wed,55.0,78.0,False,False


If `forecast` is the weather for the next six days at the specified location
then everything is working.